# Orbit Determination
##### Accurately determining the position and velocity of a satellite, as well as estimating the drag coefficient, is critical for maintaining precise orbital predictions. This notebook demonstrates the use of an Unscented Kalman Filter (UKF) to estimate these parameters from GPS NMEA measurements. By following this guide, you will be able to set up the necessary models, configure the UKF, and visualize the results to ensure the satellite remains on its predicted path.

## Jupyter Notebook Help Guide
Welcome to your Jupyter Notebook! If you need assistance or more information about any function, method, or object, the contextual help window is a valuable tool.

***Important - To have access to contextual help, start by running the [Imports](#imports) section below***

![Run Notebook](https://portal.exotrail.space/images/products/notebooks/execute_workflow.jpg)


### How to Open the Contextual Help Window


1. **Use the Help Menu**:
   - Navigate to the top menu bar and click on `Help`. From the dropdown menu, select `Show Contextual Help`.
2. **Keyboard Shortcut**:
   - Press `Shift + Tab` while your cursor is over the code you want more information on. For a more detailed view, press `Shift + Tab` twice.
3. **Using the Inspector**:
   - Type `?` followed by the function or object name and run the cell. For example, `?print`.
   - To see the full documentation, use `??` instead, e.g., `??print`.


### Tips for Using the Contextual Help


- **Inline Help**: Single press `Shift + Tab` to get a brief pop-up of the docstring.
- **Expanded Help**: Double press `Shift + Tab` or click the expand icon in the pop-up to open the full documentation in the help pane.
- **Persistent Help Pane**: Use the Help menu or `Shift + Tab` twice to dock the help pane on the right side of the screen, where it can stay open as you work.
This feature can help you understand function parameters, return types, and see example usages directly in your notebook. Happy coding!

## Imports

In [ ]:
from datetime import datetime

import matplotlib.pyplot as plt
import pandas as pd

from fds.models.determination.configuration import OrbitDeterminationConfiguration
from fds.models.determination.requests import DragCoefficientEstimationRequest
from fds.models.determination.use_case import OrbitDetermination
from fds.models.orbital_state import PropagationContext, CovarianceMatrix, OrbitalState
from fds.models.spacecraft import SpacecraftSphere
from fds.models.telemetry import TelemetryGpsNmea
from fds.models.two_line_element import TwoLineElement
from fds.utils.frames import Frame
from fds.utils.plotting import add_grid

*Note*: If you are using this notebook locally, use the following methods to configure your credentials:

```python
from fds.config import set_client_id, set_client_secret
set_client_id('CLIENT_ID')
set_client_secret('CLIENT_SECRET')
```

## Create Models

### Create Orbital State
`OrbitalState` is the object used in spacetower to group information on the initial orbit and covariance, the propagation context and the spacecraft definition.  


#### TLE

The TLE provides the orbital parameters needed for accurate satellite tracking and prediction.

In [ ]:
tle = TwoLineElement(
    "1 25544U 98067A   24142.35003124  .00022843  00000-0  38371-3 0  9995",
    "2 25544  51.6390  88.3709 0003333 191.4959 306.2513 15.51667899454382"
    )

print(f"TLE date (UTC): {tle.date}")

#### Propagation Context

The `PropagationContext` object encapsulates the settings necessary to accurately propagate the satellite's orbit by accounting for gravitational and non-gravitational perturbations. Adjusting these parameters allows for fine-tuning the balance between computational efficiency and the accuracy of the simulation.

In [ ]:
propagation_context = PropagationContext(
    model_perturbations=[
        PropagationContext.Perturbation.DRAG,
        PropagationContext.Perturbation.EARTH_POTENTIAL,
        PropagationContext.Perturbation.SRP,
        PropagationContext.Perturbation.THIRD_BODY,
    ],
    model_solar_flux=150,  # SFU
    model_earth_potential_deg=30,
    model_earth_potential_ord=30,
    model_atmosphere_kind=PropagationContext.AtmosphereModel.HARRIS_PRIESTER,
    integrator_kind=PropagationContext.IntegratorKind.DORMAND_PRINCE_853,
    integrator_min_step=0.01,  # s
    integrator_max_step=100,  # s
)

#### Spacecraft

Using the `SpacecraftSphere` class, we can define a simple spherical model.  
You can use the `SpacecraftBox` class to define more complex shapes, where battery, solar-arrays and thrusters (eletric & chemical) can be defined. 

In [ ]:
spacecraft = SpacecraftSphere(
    platform_mass=112,  # kg
    drag_coefficient=2.2,
    cross_section=0.785,
    reflectivity_coefficient=0.6,
)

#### Covariance

##### Initial state covariance matrix

The initial state covariance matrix represents the uncertainty in the spacecraft's initial position and velocity. It quantifies how precise the initial estimates are, with larger values indicating greater uncertainty.

In [ ]:
covariance = CovarianceMatrix.from_diagonal(
    diagonal=(100, 100, 100, 0.1, 0.1, 0.1),
    frame=Frame.TNW
)

##### Process noise matrix

The process noise matrix represents the variance of the position and velocity in a local orbital frame. It accounts for unmodeled forces and inaccuracies in the spacecraft's motion prediction, with larger values indicating greater uncertainty over time.

In [ ]:
process_noise_matrix = CovarianceMatrix.from_diagonal(
    diagonal=(1E-1, 1E-1, 1E-1, 1E-4, 1E-4, 1E-4),
    frame=Frame.TNW
)

#### Orbital State

Initiate the `OrbitalState` object with the TLE, the propagation context, the spacecraft, and the covariance matrix.

In [ ]:
initial_orbital_state = OrbitalState.from_tle(
    tle=tle,
    covariance_matrix=covariance,
    propagation_context=propagation_context,
    spacecraft=spacecraft
)

### Create Orbit Determination Configuration
In this section we configure the Orbit Determination to use an Unscented Kalman Filter (UKF), including parameters for the sigma points spread and outliers management.

In [ ]:
od_config = OrbitDeterminationConfiguration(
    tuning_alpha=0.5,  # Defines the spread of the sigma points. Typical values from 1E-4 to 1E-1.
    tuning_beta=2,  # Beta = 2 is optimal for Gaussian distributions.
    tuning_kappa=0,  # Defines the spread of the sigma points. Typical values range from 0 to 10.
    outliers_manager_scale=10, # The number of standard deviations for outlier detection.
    outliers_manager_warmup=0,  # The number of measurements that are processed without outlier rejection
    noise_provider_kind=OrbitDeterminationConfiguration.NoiseProviderKind.BASIC, # The type of noise provider used. Basic: the covariance matrix increase is proportional to the measurement update interval.
    process_noise_matrix=process_noise_matrix,
)

drag_estimation = DragCoefficientEstimationRequest(
    standard_deviation=0.01,
    process_noise_standard_deviation=1E-2,
)

### Create NMEA measurements
Define the GPS NMEA measurements, which include latitude, longitude, ground speed, and altitude, to be used in the orbit determination process. For more information on the GPS NMEA standard https://aprs.gids.nl/nmea/

In [ ]:
# Latitude [deg], longitude [deg], ground speed [km/s], altitude [m], geoid ondulation [m] (identically zero in this example)
nmea_measurements = [[31.945420, -126.505682,  7367.186584,  412375.054104,  0.0],
                     [29.240459, -123.511077,  7367.967736,  411821.061405,  0.0],
                     [26.459760, -120.684592,  7368.693042,  411299.690655,  0.0],
                     [23.614870, -118.003847,  7369.338715,  410835.315955,  0.0],
                     [20.715957, -115.447822,  7369.897444,  410442.560347,  0.0],
                     [17.772044, -112.997251,  7370.349873,  410156.920987,  0.0],
                     [14.791415, -110.634067,  7370.681081,  409921.752804,  0.0],
                     [11.781282, -108.341590,  7370.888166,  409812.978649,  0.0],
                     [8.748428,  -106.104106,   7370.954765,  409828.677953, 0.0],
                     [5.699253,  -103.906705,   7370.876706,  409916.755493, 0.0],
                     [2.639727,  -101.735212,   7370.647704,  410136.202027, 0.0],
                     [-0.424380,  -99.575730,  7370.267061,  410460.341200,  0.0],
                     [-3.487369,  -97.414643,  7369.738987,  410901.281852,  0.0],
                     [-6.543506,  -95.238211,  7369.066255,  411452.304177,  0.0],
                     [-9.586998,  -93.032651,  7368.253201,  412118.385775,  0.0]]

# UTC dates corresponding to the measurements
dates_ts = ["2024-05-21T08:24:02.699136",
            "2024-05-21T08:25:02.699136",
            "2024-05-21T08:26:02.699136",
            "2024-05-21T08:27:02.699136",
            "2024-05-21T08:28:02.699136",
            "2024-05-21T08:29:02.699136",
            "2024-05-21T08:30:02.699136",
            "2024-05-21T08:31:02.699136",
            "2024-05-21T08:32:02.699136",
            "2024-05-21T08:33:02.699136",
            "2024-05-21T08:34:02.699136",
            "2024-05-21T08:35:02.699136",
            "2024-05-21T08:36:02.699136",
            "2024-05-21T08:37:02.699136",
            "2024-05-21T08:38:02.699136"]

dates = [datetime.fromisoformat(ts).strftime("%Y-%m-%dT%H:%M:%S.%fZ") for ts in dates_ts]

telemetry_nmea = TelemetryGpsNmea(
    measurements=nmea_measurements,
    dates=dates,
    standard_deviation_altitude=200,
    standard_deviation_ground_speed=5,
    standard_deviation_latitude=0.002,
    standard_deviation_longitude=0.002
)

## Build & Run Use Case

In [ ]:
od_tle = OrbitDetermination(
    initial_orbital_state=initial_orbital_state,
    telemetry=telemetry_nmea,
    configuration=od_config,
    parameter_estimation_requests=[drag_estimation]
)

# Run the orbit determination
od_tle.run()

## Results/Post-Processing

### Define Post-Processing Functions
Define auxiliary functions used to plot the results

In [ ]:
# Constants for plotting
COE_KEYS = ["SMA", "ECC", "INC", "RAAN", "AOP", "TA"]
UNITS    = ["km", "", "deg", "deg", "deg", "deg"]


def extract_orbital_elements_from_orbit_list(keplerian_orbiti_list):
    return {
        COE_KEYS[0]: [keplerian_orbit.orbital_elements.SMA for keplerian_orbit in keplerian_orbiti_list],
        COE_KEYS[1]: [keplerian_orbit.orbital_elements.ECC for keplerian_orbit in keplerian_orbiti_list],
        COE_KEYS[2]: [keplerian_orbit.orbital_elements.INC for keplerian_orbit in keplerian_orbiti_list],
        COE_KEYS[3]: [keplerian_orbit.orbital_elements.RAAN for keplerian_orbit in keplerian_orbiti_list],
        COE_KEYS[4]: [keplerian_orbit.orbital_elements.AOP for keplerian_orbit in keplerian_orbiti_list],
        COE_KEYS[5]: [keplerian_orbit.orbital_elements.TA for keplerian_orbit in keplerian_orbiti_list]
    }


def plot_keplerian_element(dates_list, rel_times, osculating_states, mean_states, element, title):
    plt.figure()
    plt.plot(rel_times, osculating_states[element], 'o', label="Osculating")
    plt.plot(rel_times, mean_states[element], 'o', label="Mean")
    plt.title(title)
    add_grid(plt.gca())
    plt.xlabel(f"Time [h] since {dates_list[0]}")
    plt.ylabel(f"{title} [{UNITS[COE_KEYS.index(element)]}]")
    plt.legend()
    plt.show()


def plot_residuals(dates_list, rel_times, residuals_statistics):
    n_subplots = 4
    fig, axs = plt.subplots(n_subplots, 1, sharex=True)
    for i, (_k, _v) in enumerate(zip(["altitude", "ground_speed", "latitude", "longitude"], 
                   [residuals_statistics.altitude, residuals_statistics.ground_speed,
                    residuals_statistics.latitude, residuals_statistics.longitude])):
        axs[i].plot(rel_times, _v.values, label='Residuals')
        # Add a red dashed line at 3 and -3
        axs[i].axhline(y=3, color='r', linestyle='--', label=r'$\pm3$')
        axs[i].axhline(y=-3, color='r', linestyle='--')
        axs[i].set_ylabel(_k)
    for ax in axs:
        add_grid(ax)
    axs[0].legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)
    plt.suptitle("Normalized Residuals")
    plt.xlabel(f"Time [h] since {dates_list[0]}")
    fig.tight_layout()
    plt.show()


### Extract Results

In [ ]:
results = od_tle.result

# Extract estimated states
dates = results.in_depth_results.dates
relative_times = [(date - dates[0]).total_seconds() / 3600 for date in dates]
estimated_osc_states = extract_orbital_elements_from_orbit_list(results.in_depth_results.osculating_orbits)
estimated_mean_states = extract_orbital_elements_from_orbit_list(results.in_depth_results.mean_orbits)

# Extract residual statistics
residual_statistics = results.report.residuals_statistics
columns = ["Parameter", "Mean", "Median", "Std", "Max", "Min"]
data = []
for name, v in zip(["altitude", "ground_speed", "latitude", "longitude"], 
                   [residual_statistics.altitude, residual_statistics.ground_speed,
                    residual_statistics.latitude, residual_statistics.longitude]):
    data.append([name, v.mean, v.median, v.standard_deviation, v.max, v.min])

residual_statistics_df = pd.DataFrame(data, columns=columns)
print("Normalized residuals statistics")
residual_statistics_df

### Plot Results

##### Estimated Keplerian orbital elements, osculating and mean

In [ ]:
for el in COE_KEYS:
    plot_keplerian_element(dates, relative_times, estimated_osc_states, estimated_mean_states,el, el)

##### Normalized residuals

In [ ]:
plot_residuals(dates, relative_times, results.report.residuals_statistics)

##### Estimated drag coefficient

In [ ]:
plt.figure()
plt.plot(relative_times, [elem.value for elem in results.in_depth_results.estimated_drag_coefficients], 'o')
plt.title("Estimated Drag Coefficient")
plt.xlabel(f"Time [h] since {dates[0]}")
plt.ylabel("Drag Coefficient")
add_grid(plt.gca())

plt.show()